In [15]:
import os
import random
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [6]:
train_data = pd.read_csv("./data/train_images.csv")
train_data['image_path'] = 'data' + train_data['image_path']

train_df, test_df = train_test_split(train_data, test_size=0.2, random_state=42)

print(f"Train: {len(train_df)}, Test: {len(test_df)}")

Train: 3140, Test: 786


In [7]:
class BirdieDataset(Dataset):
    def __init__(self, dataframe, root_dir, transform=None):
        self.data = dataframe
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        img_path = os.path.join(self.root_dir, self.data.iloc[index, 0])
        image = Image.open(img_path).convert("RGB")
        # 0-based indexing
        label = int(self.data.iloc[index, 1]) - 1  
        
        if self.transform:
            image = self.transform(image)
        return image, label

In [8]:
# tranformations
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [9]:
train_dataset = BirdieDataset(train_df, root_dir="", transform=train_transform)
test_dataset = BirdieDataset(test_df, root_dir="", transform=val_test_transform)

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [10]:
model = models.efficientnet_b0(pretrained=True)
model.classifier[1] = nn.Linear(model.classifier[1].in_features, 200) 

/Users/meryjoy/opt/anaconda3/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/meryjoy/opt/anaconda3/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [11]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [12]:
def train_model(model, train_loader, val_loader, criterion, optimizer, epochs=10):
    best_acc = 0.0
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for images, labels in tqdm(train_loader):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

        train_acc = 100. * correct / total
        val_acc = evaluate_model(model, val_loader)

        print(f"Epoch {epoch+1}/{epochs} - Loss: {running_loss:.4f} - Train Acc: {train_acc:.2f}% - Val Acc: {val_acc:.2f}%")
        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), "best_model.pth")
    print("Training Complete. Best Validation Accuracy:", best_acc)

In [16]:
def evaluate_model(model, loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    return 100. * correct / total

# Train the model
train_model(model, train_loader, test_loader, criterion, optimizer, epochs=10)

100%|█████████████████████████████████████████████| 99/99 [15:12<00:00,  9.22s/it]


Epoch 1/10 - Loss: 512.7622 - Train Acc: 2.55% - Val Acc: 9.03%


100%|█████████████████████████████████████████████| 99/99 [14:07<00:00,  8.56s/it]


Epoch 2/10 - Loss: 449.0033 - Train Acc: 16.02% - Val Acc: 21.88%


100%|█████████████████████████████████████████████| 99/99 [14:04<00:00,  8.53s/it]


...

Epoch 8/10 - Loss: 158.7963 - Train Acc: 68.50% - Val Acc: 53.18%


100%|██████████████████████████████████████████| 99/99 [2:55:23<00:00, 106.30s/it]


Epoch 9/10 - Loss: 134.4293 - Train Acc: 73.50% - Val Acc: 55.22%


100%|█████████████████████████████████████████████| 99/99 [13:25<00:00,  8.13s/it]


Epoch 10/10 - Loss: 112.6465 - Train Acc: 79.24% - Val Acc: 56.36%
Training Complete. Best Validation Accuracy: 56.36132315521628
